# U-Net Throughput Benchmark

This notebook measures the training throughput (images/second) of the real and complex U-Nets as a function of batch size and model width. Synthetic inputs are used so the results isolate compute efficiency on the current hardware.

In [1]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

from pathlib import Path
import time
import itertools

import torch
import pandas as pd

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == 'notebooks':
    PROJECT_ROOT = PROJECT_ROOT.parent
SRC_ROOT = PROJECT_ROOT / 'src'
import sys
if str(SRC_ROOT) not in sys.path:
    sys.path.append(str(SRC_ROOT))

from models.real_unet import RealUnet
from models.cx_unet import ComplexUnet

torch.manual_seed(0)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
DEVICE

device(type='mps')

In [2]:
BATCH_SIZES = [1, 2, 4]
FEATURE_CONFIGS = {
    'tiny': [8, 16, 32, 64, 128],
    #'base': [16, 32, 64, 128, 256],
    #'large': [32, 64, 128, 256, 512],
}
BENCH_STEPS = 10  # iterations measured
WARMUP_STEPS = 5
INPUT_SHAPE = (1, 640, 320)  # (channels, H, W)
print(f"Device: {DEVICE}")
print(f"Benchmark steps: {BENCH_STEPS}, warmup: {WARMUP_STEPS}")

Device: mps
Benchmark steps: 10, warmup: 5


In [3]:
def make_model(kind: str, features: list[int]):
    if kind == 'real':
        return RealUnet(in_channels=1, out_channels=1, features=features, width_scale=1.0).to(DEVICE)
    if kind == 'complex':
        return ComplexUnet(in_channels=1, out_channels=1, features=features).to(DEVICE)
    raise ValueError(f"Unknown model kind: {kind}")


def make_batch(batch_size: int):
    shape = (batch_size, *INPUT_SHAPE)
    real = torch.randn(shape, dtype=torch.float32, device=DEVICE)
    imag = torch.randn(shape, dtype=torch.float32, device=DEVICE)
    batch = torch.complex(real, imag)
    return batch


def benchmark_step(model, optimizer, batch_size: int):
    inputs = make_batch(batch_size)
    targets = make_batch(batch_size)
    # simple L1 loss to match training behavior
    criterion = lambda pred, tgt: torch.mean(torch.abs(pred - tgt))

    # warmup
    for _ in range(WARMUP_STEPS):
        optimizer.zero_grad(set_to_none=True)
        pred = model(inputs)
        loss = criterion(pred, targets)
        loss.backward()
        optimizer.step()

    torch.cuda.synchronize() if DEVICE.type == 'cuda' else None
    start = time.perf_counter()
    for _ in range(BENCH_STEPS):
        optimizer.zero_grad(set_to_none=True)
        pred = model(inputs)
        loss = criterion(pred, targets)
        loss.backward()
        optimizer.step()
    torch.cuda.synchronize() if DEVICE.type == 'cuda' else None
    elapsed = time.perf_counter() - start
    images = batch_size * BENCH_STEPS
    return images / elapsed

In [4]:
results = []
for model_kind, config_name in itertools.product(['real', 'complex'], FEATURE_CONFIGS.keys()):
    features = FEATURE_CONFIGS[config_name]
    print(f"Benchmarking {model_kind} | config {config_name}")
    model = make_model(model_kind, features)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    for batch_size in BATCH_SIZES:
        try:
            imgs_per_sec = benchmark_step(model, optimizer, batch_size)
        except RuntimeError as err:
            imgs_per_sec = float('nan')
            print(f"  batch {batch_size} OOM/error: {err}")
        results.append({
            'model': model_kind,
            'config': config_name,
            'features': features,
            'batch_size': batch_size,
            'images_per_sec': imgs_per_sec,
        })
    del model, optimizer
    torch.cuda.empty_cache() if DEVICE.type == 'cuda' else None

df = pd.DataFrame(results)
df

Benchmarking real | config tiny
Benchmarking complex | config tiny


/opt/anaconda3/envs/ece570/lib/python3.12/site-packages/torch/nn/modules/module.py:1341: UserWarning: Complex modules are a new feature under active development whose design may change, and some modules might not work as expected when using complex tensors as parameters or buffers. Please file an issue at https://github.com/pytorch/pytorch/issues/new?template=bug-report.yml if a complex module does not work as expected.
  warnings.warn(
/Users/giodegeronimo/Desktop/ECE570/ece570-tinyreproductions/src/models/cx_unet.py:143: UserWarning: The operator 'aten::_linalg_eigh.eigenvalues' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:15.)
  eigvals, eigvecs = torch.linalg.eigh(V)  # (C, 2), (C, 2, 2)


,model,config,features,batch_size,images_per_sec
0,real,tiny,"[8, 16, 32, 64, 128]",1,34.318817
1,real,tiny,"[8, 16, 32, 64, 128]",2,64.119999
2,real,tiny,"[8, 16, 32, 64, 128]",4,89.939659
3,complex,tiny,"[8, 16, 32, 64, 128]",1,1.008742
4,complex,tiny,"[8, 16, 32, 64, 128]",2,1.040376
5,complex,tiny,"[8, 16, 32, 64, 128]",4,1.076109


In [5]:
df.pivot_table(index=['model', 'config'], columns='batch_size', values='images_per_sec')

,batch_size,1,2,4
model,config,,,
complex,tiny,1.008742,1.040376,1.076109
real,tiny,34.318817,64.119999,89.939659
